# Clase 17: Introducción al Aprendizaje Supervisado


**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Matías Rojas**

## Objetivos de la clase: 
   
- Introducir al aprendizaje supervisado mediante el uso de ejemplos aplicados.
- Entender el framework utilizado para resolver la tarea de clasificación y reforzar los contenidos asociados a la ingeniería de características.
- Estudiar las métricas de evaluación más comunes para la tarea seleccionada.
- Entender la idea de Holdout y K-Fold para evaluar modelos.
- Experimentar con los primeros modelos de clasificación del curso.


## Panorama General Hasta el Momento

<div align='center'>
<img src='https://i.ibb.co/DRvgXs6/etapas.png'/>
</div>

---

<br>

<div align='center'>
<img src="https://i.ibb.co/BT3Dt2L/machine-learning.png" alt="Panorama General ML: Clasificación supervisada, No supervisada y Aprendizaje Reforzado binario" width=700/>
</div>


El aprendizaje automático es el estudio de algoritmos que automáticamente mejoran su rendimiento a través de la experiencia. Estos algoritmos construyen modelos basados en datos de muestra con la intención de realizar predicciones sin ser explícitamente programados para hacerlo.

## Aprendizaje Supervisado

El aprendizaje supervisado se basa en trabajar con datasets cuyas observaciones son **características** que describen a algún objeto. Estas observaciones además cuentan con una **etiqueta/valor real**, la cuál corresponde a una clase o valor que se le asigna a cada observación.

Cuando el valor a predecir es un(a): 

- Categoría/Etiqueta, el problema que se resuelve se denomina **Clasificación**.
- Valor real, el problema que se resuelve se denomina **Regresión**.


En otras palabras, las etiquetas pertenecen a un número finito de clases. Por ejemplo, en caso que estemos describiendo a una persona, el vector asociado a cada observación puede contener los siguentes **features/características**:

- su altura en cm, 
- edad, 
- peso en kg, 
- residencia, 
- etc...

Mientras que la **etiqueta** puede ser si la persona *quiere o no contratar un servicio de internet*, es decir un valor boolenao, $\{ True, False\}$ 

In [ ]:
import pandas as pd

# Ejemplo de un dataset con una etiqueta con un conjunto de datos discreto (clasificación).
pd.DataFrame(
    [[177, 43, 72, "Maipú", True], [160, 16, 60, "Pudahuel", False]],
    columns=["Altura", "Edad", "Peso", "Residencia", "Posible cliente?"],
)

Como también lo que está dispuesto a gastar en el plan.

In [ ]:
# Ejemplo de un dataset con una etiqueta con un conjunto de datos continuo (regresión).
pd.DataFrame(
    [[177, 43, 72, "Maipú", 55000], [160, 16, 60, "Pudahuel", 0]],
    columns=["Altura", "Edad", "Peso", "Residencia", "Cuánto está dispuesto a pagar?"],
)

Dado un conjunto de datos etiquetados, el objetivo del aprendizaje supervisado es crear algoritmos/modelos que permitan **asignar de forma automática categorías o valores a observaciones nuevas**. 

En términos prácticos, dada una nueva observación representada por su vector de características, el modelo generado debe ser capaz de asignar una etiqueta a dicha observación.

### Framework General de Aprendizaje Supervisado Clásico

La siguiente lista muestra las etapas que debería cumplir un algoritmo de aprendizaje supervisado clásico (i.e., no red neuronal)

1. **Feature Engineering y Preprocesamiento**: Recolectar y preparar los datos.
2. **Entrenar** un algoritmo de clasificación/regresión usando los datos.
3. **Evaluar** qué tan bien el clasificador puede clasificar nuevos ejemplos.
4. **Optimizar los modelos** modificando sus hiperparámetros.

### Feature Engineering

Feature Engineering/Ingeniería de Características es la etapa en que se transforman los "datos raw/crudos" de entrada en un dataset, de manera que se tengan datos robustos para ser usados por el clasificador/regresor que desean entrenar.


Este proceso incluye: 

- **Creación de nuevas Features** a partir de operaciones usando los datos disponibles.
- **Transformaciones** como las vistas en la clase de preprocesamiento (escalamiento, normalización, one hot encoding para variables categóricas etc...).
- **Reducción de Características** en la que se combinan/reducen características redundantes (usando por ejemplo, PCA).
- **Selección de Características** en la que a partir de diversos criterios se seleccionan las características que más aportan al modelo.

El proceso de generar y preprocesar las features requiere mucha creatividad y al mismo conocimiento del dominio del problema.

La creación de nuevas características puede incluir: 

- **Generación de features a partir de la combinación de otras**: Sumar, restar, multiplicar y contar distintas features puede agregar más información que ellas por si mismas. Esto también incluye el preprocesar features independientemente con funciones no linales como $\log$, $\exp$, etc...
- **Discretización de una variable numérica a través de Binning**: Transformar una variable numérica a una variable categórica según rangos usando bins o percentiles. Recordar el uso de los métodos `cut` y `qcut`. Ver también `Binarizer` y `KBinsDiscretizer` en `sklearn.preprocessing`.
- **Clusters** generados a partir de las features (no incluir las labels! es la información que quieren predecir)
- **Bag-of-words para texto**. Técnica similar a One Hot encoding en donde cada palabra es una columna y se cuenta la cantidad de apariciones de cada palabra en una oración.
- **Seno/Coseno** para codificar variables cíclicas, como las horas, días, meses o años. Por ejemplo, para los días de la semana en donde $p \in \{1,2,3,4,5,6,7\}$, se pueden generar dos features usando 
$$
p_{sin} = \sin{\frac{2 \times \pi \times p}{\max{p}}} 
$$
y
$$
p_{cos} = \cos{\frac{2 \times \pi \times p}{\max{p}}} 
$$
- **Datos Temporales**: La idea aquí es transformar toda la secuencia temporal a un vector que la describa. Para esto, se pueden calcular descriptores como media, moda, tiempo entre valles, picos, diferencias entre valles y picos en un determinado tiempo, etc...  .Una librería útil para esto es tsfel: https://tsfel.readthedocs.io/en/latest/
- **Transformaciones polinomiales a variables numéricas**: Esto se basa en que en dimensiones más altas/no lineales los datos pueden mostrar patrones que no se presentan en los datos originales y que pueden ser aprendidas por el algoritmo. Ver la transformación `PolynomialFeatures` de `sklearn.preprocessing`.

Una feature buena cumple que: 

- **Tiene un alto poder predictivo**
- **Computabilidad rápida** 
- **No correlación con otras features**

Este proceso debe ser determinista ya que al momento de predecir datos nuevos, las transformaciones y features calculadas sobre estos deben ser las mismas que las utilizadas en el proceso de entrenamiento. Para solucionar esto, es muy recomendable usar los `Pipelines`.

----



El modelo construido debe **generalizar**, es decir, debe ser capaz de realizar predicciones correctas en nuevas observaciones. Para esto es útil pensar que el modelo generado está separando los datos por clases a través de un *decision boundary*. Mientras más holgado sea este *decision boundary*, mejor podrá generalizar el modelo.

<div align='center'>
<img src='https://i.ibb.co/3mcx35c/overfitting.png' witdh=400/>

</div>

<div align='center'>
<a href='https://www.researchgate.net/figure/Example-of-overfitting-in-classification-a-Decision-boundary-that-best-fits-training_fig1_349186066'>Ejemplo de *Overfitting* en researchgate</a>
</div>
    
<br/>
 
### Cómo determinar que algorimo utilizar 
 
Muchas veces se piensa que lo más importante es la **capacidad predictiva** del modelo.
Sin embargo, también hay otros factores muy relevantes que determinarán que algoritmo predictivo utilizar: 

**Eficiencia**: 
  - ¿Qué tanto se está demorando mi modelo en entrenar? 
  - ¿Y en predecir? 
  - ¿Es eficiente en memoria? 
  - ¿Debe almacenar el dataset de entrenamiento para funcionar?
  - ¿Es posible usarlo en tiempo real para algún tipo de solución online?
  
**Número de Features y Ejemplos Requeridos**: 
  - ¿Cuántos datos o features son requeridos para entrenar el modelo?
  - ¿Es compatible con la cantidad que dispongo?
  - ¿El tipo de features (i.e., categorícas, numéricas, combinación de ambas, etc...) es compatible con el algoritmo?
  
**Explicabilidad**: 
  - ¿Puedo explicar por qué el modelo está clasificando/regresionando de la manera que lo hace? 
  
***Fairness***: 
  - ¿Mi modelo es injusto con respecto a algún grupo social?


### ¿Cómo saber si un modelo es bueno o no?

Resumimos la capacidad predictiva de un modelo mediante **métricas de desempeño** (performance metrics).

Las métricas se calculan contrastando los valores predichos versus los valores reales de la variable objetivo (con datos no usados durante entrenamiento)

##  Matriz de Confusión

<div align='center'>
    <img src="https://i.ibb.co/5sMqPDR/matriz-conf.png" alt="Ejemplo de una matriz de confusión para un problema de clasificación binario" width=450/>
</div>

<center>Ejemplo de una matriz de confusión para un problema de clasificación binario.</center>



---

> Ejemplo: Alergia a cierto medicamento en donde la clase `+` indica alergia.


Nuestro dataset tiene 10.000 observaciones distribuidos de la siguiente forma:

- Clase `+`: 100 observaciones.
- Clase `-`: 9900 observaciones.


Luego, creamos un modelo que clasificó nuestro dataset y graficamos sus resultados a través de la siguiente matriz de confusión:


|                    | **Predicha (`+`)**  | **Predicha (`-`)** |
|--------------------|---------------------|--------------------|
| **Real (`+`)**     | 10                  | 90                 |
| **Real (`-`)**     | 100                 | 9800               |

---

> **Pregunta**: ¿Cuales métricas de desempeño conocen para evaluar este caso?

### Métricas de desempeño

Métricas basadas en contar datos correcta e incorrectamente clasificados:

- **Accuracy (Exactitud)**: $$\text{accuracy} = \frac{\text{número de predicciones correctas}}{\text{número de predicciones totales}}$$

- **Error rate (Tasa de error)**: $$\text{error rate} = \frac{\text{número de predicciones incorrectas}}{\text{número de predicciones totales}}$$



- En nuestro ejemplo anterior: 

$$\text{accuracy} = \frac{9810}{10000} = 0.981$$

$$\text{error rate} = \frac{190}{10000} = 0.019$$


> **Pregunta ❓:** ¿Cuál es el problema de `Accuracy` en nuestro ejemplo?



---


#### Métricas Basadas en la Matriz de Confusión

Una posible solución a este problema son las métricas basadas en la matriz de confusión:

<div align='center'>
    <img src="https://i.ibb.co/5sMqPDR/matriz-conf.png" alt="Ejemplo de una matriz de confusión para un problema de clasificación binario" width=450/>
</div>

- **Precision**:  Fracción de ejemplos correctamente predichos como `+` con respecto a todos los predichos `+`.

$$P = \frac{\text{Clasificados correctamente como positivo}}{\text{Todos los predichos como positivos}} =\frac{TP}{(TP + FP)}$$


<br>

- **Recall**: Fracción de ejemplos `+` que son correctamente clasificados: 

$$R = \frac{\text{Clasificados correctamente como positivo}}{\text{Todos los que debería haber clasificado como positivos}}  = \frac{TP}{(TP+FN)}$$


<br>

- **F1 measure**: Combina precisión y recall usando una media armónica (i.e., media que castiga si ambos valores son muy diferentes).

$$F = \frac{2PR}{(P+R)}$$



|                    | **Predicha (`+`)**  | **Predicha (`-`)** |
|--------------------|---------------------|--------------------|
| **Real (`+`)**     | 10                  | 90                 |
| **Real (`-`)**     | 100                 | 9800               |

En nuestro ejemplo anterior:


$$P = \frac{10}{110} = 0.\bar{09}$$

$$R = \frac{10}{100} = 0.1$$

$$F = \frac{2 \cdot 0.1 \cdot 0.\bar{1}}{(0.1 + 0.\bar{1})} \approx 0.095$$ 

Ahora claramente se nota el problema.

In [ ]:
p = 10 / 110
r = 10 / 100

f = 2 * p * r / (p + r)
f

#### Matriz de confusión multiclase

Cuando tenemos $k$ clases, la matriz de confusión es una matriz de $k \times k$.

<div align='center'>
<img src="https://i.ibb.co/Z2Strv9/matriz-conf-multiclase.png" alt="Ejemplo de una matriz de confusión para un problema de clasificación binario" style="width: 500px;"/>
</div>

¿Cómo calculamos las métricas?


#### Métricas de Desempeño Generalizadas: 


- Precision: Fracción de ejemplos asignados a la clase `i` que son realmente de la clase `i`.

$$\text{precision} = \frac{c_{ii}}{\sum_{j}c_{ji}}$$


- Recall: Fracción de ejemplos de la clase `i` correctamente clasificados: 

$$\text{recall} = \frac{c_{ii}}{\sum_{j}c_{ij}}$$

- Accuracy: Fracción de ejemplos correctamente clasificados:

$$\text{accuracy} = \frac{\sum_{i}c_{ii}}{\sum_{j}\sum_{i}c_{ij}}$$

#### Estrategia de Agregación

- **Macroaveraging**
    - Computar métrica para cada clase y luego promediar. 
    - Sobrerepresentan clases minoritarias al tratar a todas por igual.

- **Weighted**
    - Computar métrica para cada clase y luego hace un promedio ponderado por el número de ejemplos de esa clase.
    - Al ser ponderado por el número de casos, da más prioridad a las clases frecuentes.


`Scikit` provee un acceso rápido a todas estas métricas a través de su función `sklearn.metrics.classification_report`

### Underfitting y Overfitting

Errores de entrenamiento o **Underfitting**. 
- Malos resultados sobre los datos de entrenamiento
- El clasificador no tiene capacidad de aprender el patrón.

Errores de generalización o **Overfitting**. 
- Malos resultados sobre datos nuevos 
- El modelo se hace demasiado específico a los datos de entrenamiento. 


<div align='center'>
<img src='https://i.ibb.co/Sc7SBs2/tipos-fit.png' width=800/>
</div>

<div align='center'>
    Fuente: The Hundred-Page Machine Learning Book.
</div>

## Nuestro problema de hoy: Pingüinos  🐧


Origen del dataset:

**Palmer Archipelago (Antarctica) penguin data**: 

*Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network.*

https://github.com/allisonhorst/palmerpenguins

![Pinguinos](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/man/figures/lter_penguins.png)


    
    


### Atributos
 
- `culmen_length_mm`: Largo del culmen (vértice o borde superior de la mandíbula)  (mm).
- `culmen_depth_mm`: Alto del culmen (vértice o borde superior de la mandíbula) (mm).
- `flipper_length_mm`: Longitud de las aletas (mm).
- `body_mass_g`: Masa corporal (g).
- `island`: Isla de origen (Dream, Torgersen, or Biscoe) en el archipiélago de Palmer (Antarctica).
- `sex`: Sexo del pinguino.

![Detalle Variables](https://allisonhorst.github.io/palmerpenguins/reference/figures/culmen_depth.png)
    
<center>Créditos a Allison Horst por sus excelentes ilustraciones https://github.com/allisonhorst </center>    
    
    
### Variable a predecir

- `species`: Especie del pinguino (Chinstrap, Adélie, or Gentoo)

## Exploración y Preprocesamiento


In [ ]:
# Instalar graphviz para visualizar el árbol generado
# Deben instalar antes graphviz: https://www.graphviz.org/download/

import sys

!pip install graphviz

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("penguins.csv").dropna().reset_index(drop=True)
df

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(
    df, dimensions=df.iloc[:, 2:].columns, color="species", height=1000
)
fig.show()

In [ ]:
px.parallel_categories(df, dimensions=["island", "species"])

In [ ]:
df.head(2)

---

## Holdout

Consiste en particionar nuestro dataset en conjuntos de:

- **Training**: conjunto que se utiliza para **entrenar** el modelo.
- **Testing**: datos que se usa para **evaluar** qué tan bien predice el modelo (a través de las métricas de evaluación). 


Comunmente se dividen en proporción $2/3$ y $1/3$ del dataset respectivamente. Sin embargo, todo depende de la cantidad de datos que se posean: si se tiene millones de ejemplos, quizas puede dividirse en 95% train, 5% test sin problemas. 


La evaluación puede variar mucho según las particiones escogidas: 

- Training pequeño -> modelo sesgado, 
- Testing pequeño -> evaluación poco confiable.


Esta ténica se puede **Random Subsampling** para seleccionar aleatoriamente las observaciones de cada uno de estos conjuntos.

Para ejecutar todo esto usaremos `train_test_split`. Veamos algunos de sus parámetros:

- `test_size = 0.33` - indica el tamaño del test de evaluación.
- `shuffle = True` - indica que ejecutaremos Random Subsampling.
- `stratify = labels` - intenta manetener la distribución de clases original en ambos conjuntos.


### Validation set:

Cuando se desea realizar una búsqueda de los mejores algoritmos y sus hiperparámetros, el dataset puede ser dividido en 3:


- **Training**: Se utiliza para entrenar los modelos.
- **Validation**: Se utiliza para seleccionar el mejor modelo al ir variando sus hiperparámetros.
- **Testing**: Se utiliza para evaluar el modelo previo a ser entregado o puesto en producción. Esta evaluación solo se hace sobre el modelo final.


En este caso la división puede ser $70\%, 15\%, 15\%$ respectivamente.

In [ ]:
# Holdout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

features = df.drop(columns=["species"])
labels = df.loc[:, "species"]


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# distribución original
labels.value_counts() / labels.count() * 100

In [ ]:
# conjunto de entrenamiento
y_train.value_counts() / y_train.count() * 100

In [ ]:
# conjunto de pruebas
y_test.value_counts() / y_test.count() * 100



### Otra Opción: `cross-validation`

Se particiona el dataset en k conjuntos disjuntos o folds:

---
    Para cada partición i:
        - Juntar todas las k-1 particiones restantes y entrenar el modelo sobre esos datos.
        - Evaluar el modelo en la partición i.
        
    El error total = suma de errores de todos los modelos  

---

<img src='https://i.ibb.co/Sc7SBs2/tipos-fit.png' width=400>


Veremos más de esto en las próximas clases

### Preprocesamiento y Data Leakage


Data Leakage o fuga de datos se refiere al uso de datos de prueba dentro del entrenamiento de un modelo predictivo (lo que ovbiamente es incorrecto).

Es muy importante que el **preprocesamiento y feature engineering lo hagan siempre sobre los datos de entrenamiento y no sobre todo el dataset**. De lo contrario, estarían ocupando datos destinados a evaluar para entrenar el modelo (o el preprocesamiento) lo que puede inducir a resultados muy buenos cuando en verdad no deberían serlos.


Mas información en [data-leakage de scikit-learn](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler

In [ ]:
ct = ColumnTransformer(
    [
        (
            "Scaler",
            RobustScaler(),
            [
                "culmen_length_mm",
                "culmen_depth_mm",
                "flipper_length_mm",
                "body_mass_g",
            ],
        ),
        ("OneHot", OneHotEncoder(sparse=False), ["island", "sex"]),
    ]
)

In [ ]:
X_train_preprocessed = pd.DataFrame(
    ct.fit_transform(X_train),
    columns=np.concatenate(
        [ct.transformers_[0][2], ct.transformers_[1][1].get_feature_names()], axis=0
    ),
)

X_train_preprocessed

---

## Árboles de Decisión

Árboles que fragmentan el dataset en condiciones.

- Nodo raíz: Sin arcos entrantes, 2 o más salientes. Contienen una condición sobre alguna feature.
- Nodo interno: 1 arco entrante, 2 o más salientes.  Contienen una condición sobre alguna feature.
- Nodo hoja/terminal: 1 arco entrante, nunguno saliente. Indican la clase.


Nota: `tree` de `Scikit-learn` solo implementa 2 ramas salientes en los nodos raíz y interno.

Se entrenan recursivamente:

Estrategia: Top down (greedy) - Divide y vencerás:

---
    Seleccionar un atributo para el nodo raíz y crear rama para cada valor posible del atributo.
    Luego: dividir las instancias del dataset en subconjuntos, uno para cada rama que se extiende desde el nodo.
    Por último: repetir de forma recursiva para cada rama, utilizando sólo las instancias que llegan a ésta.

---

In [ ]:
import graphviz
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, export_graphviz

tree_pipe = Pipeline(
    [("preprocesamiento", ct), ("tree", DecisionTreeClassifier(criterion="entropy"))]
)

# noten que aquí se pasa X_train ya que la etapa de
# preprocesamiento está incluida en el pipeline (primera etapa)

tree_pipe = tree_pipe.fit(X_train, y_train)

In [ ]:
y_pred = tree_pipe.predict(X_test)
y_pred

### Evaluación

In [ ]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

### Visualizar el árbol

In [ ]:
# con esto obtenemos los nombres de las columnas
# en el primer elemento del arreglo de obtienen las numéricas a partir
# de la lista de la primera transformación y en la segunda se obtienen a partir
# de las columnas generadas por el one hot encoding.
cols_names = np.concatenate(
    [
        tree_pipe.steps[0][1].transformers_[0][2],
        tree_pipe.steps[0][1].transformers_[1][1].get_feature_names(),
    ]
)
cols_names

Para ejecutar la siguiente celda necesitarán tener instalado Graphviz:

https://graphviz.org/download/

In [ ]:
dot_data = export_graphviz(
    tree_pipe.steps[1][1],
    out_file=None,
    feature_names=cols_names,
    class_names=labels.unique(),
    filled=True,
    rounded=True,
    special_characters=True,
)
graph = graphviz.Source(dot_data)
graph

> **Pregunta ❓**: ¿Cómo eligo los atributos y sus divisiones? 

- La idea es ir dividiendo el dataset en nodos a la vez que se crea el árbol más pequeño posible.
- Heurística: escoge el atributo cuya división que produce nodos lo más “puros” posibles (es decir, que pertenezcan mayoritariamente a la misma clase). El criterio comunmente utilizado es Information Gain.


### Resumen Árboles de decisión



| Ventajas                                | Deseventajas                                                                   |
|-----------------------------------------|--------------------------------------------------------------------------------|
| Simple de entender y interpretar.       | Se deben preprocesar las variables categóricas y ordinales antes de ser usadas |
| Pueden ser visualizados.                | No escala tan bien a muchas decisiones (crece mucho el árbol)                  |
| Al ser árbol, es muy rápido de evaluar. |                                                                                |

## Extra: Guía de Scikit-learn para Elegir el Modelo

<img src='https://i.ibb.co/1XkHvRC/ml-map.png' />